1.  **Logistic Regression** 常用于 二分类 的场合，
2. **Softmax Regression** (又名 Multinomial Logistic Regression) 就是 Logistic Regression 在 多类别 场合下的推广，这种推广并不需要训练或者结合多个二分类分类器。


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import time

In [2]:
input_data

<module 'tensorflow.examples.tutorials.mnist.input_data' from '/opt/conda/lib/python3.6/site-packages/tensorflow/examples/tutorials/mnist/input_data.py'>

In [3]:
mnist = input_data.read_data_sets('./data',one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [4]:
os.listdir('./data/')

['t10k-images-idx3-ubyte.gz',
 'train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

In [5]:
learning_rate = 0.1
batch_size = 128
n_epochs = 30

In [6]:
x = tf.placeholder(tf.float32, shape=(batch_size, 784), name='placeholder_x')
y = tf.placeholder(tf.int32, shape=(batch_size, 10), name='placeholder_y')

In [7]:
w = tf.Variable(tf.random_normal(shape=(784,10), mean=0.0, stddev=0.01), name='weights')
b = tf.Variable(tf.zeros(shape=(10)), name='bias') # Or shape=(1, 10), both ok

In [8]:
logits = tf.matmul(x,w)+b

In [9]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y,name='cross_entropy_loss')
loss = tf.reduce_mean(cross_entropy)

In [10]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [11]:
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    merged_summaries = tf.summary.merge_all()

In [12]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./MNIST_LR', sess.graph)
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples / batch_size) # 55000/128 = 429.6875 ->int 429
    
    for i in range(n_epochs):
        total_loss = 0.0
        
        for j in range(n_batches):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            # 该循环的每个步骤中，会随机抓取训练数据中的100个批处理数据点, 随机训练（stochastic training）
            _, loss_batch = sess.run([optimizer, loss], feed_dict={x:x_batch,y:y_batch})
            total_loss += loss_batch
        
        print('Mean Loss for epoch_{}, {}'.format(i, total_loss/n_batches))
    
    print('Total running time: {}'.format(time.time() - start_time))
    print('Finished Training')
    
    # Test model
    y_pred_test = tf.nn.softmax(logits=logits)
    correct_pred = tf.equal(tf.argmax(y_pred_test,1), tf.argmax(y,1)) # 1：按行计算，找出最大值的index
    count_correct_pred = tf.reduce_sum(tf.cast(correct_pred, tf.float32))
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_pred = 0
    for i in range(n_batches):
        x_batch,y_batch = mnist.test.next_batch(batch_size)
        count_correct_pred_per_batch = sess.run(count_correct_pred, feed_dict={x:x_batch,y:y_batch})
        total_correct_pred += count_correct_pred_per_batch
    print('Accuracy on TEST set: ',total_correct_pred/mnist.test.num_examples)
          
    writer.close()
          

Mean Loss for epoch_0, 0.5904954635041021
Mean Loss for epoch_1, 0.3814849774320642
Mean Loss for epoch_2, 0.3492606397205855
Mean Loss for epoch_3, 0.3314374920093652
Mean Loss for epoch_4, 0.32027248833285066
Mean Loss for epoch_5, 0.311461668748122
Mean Loss for epoch_6, 0.3062028355531759
Mean Loss for epoch_7, 0.3014088851032835
Mean Loss for epoch_8, 0.2976602988018023
Mean Loss for epoch_9, 0.29298091191789766
Mean Loss for epoch_10, 0.2918372745766784
Mean Loss for epoch_11, 0.2886862069507301
Mean Loss for epoch_12, 0.28457754162641674
Mean Loss for epoch_13, 0.2847773392092098
Mean Loss for epoch_14, 0.28349766246366614
Mean Loss for epoch_15, 0.2796896083024276
Mean Loss for epoch_16, 0.27927566585424063
Mean Loss for epoch_17, 0.2776003909451422
Mean Loss for epoch_18, 0.2767240896925226
Mean Loss for epoch_19, 0.2764255972036393
Mean Loss for epoch_20, 0.27306254454703877
Mean Loss for epoch_21, 0.2731436590700994
Mean Loss for epoch_22, 0.27238364126924036
Mean Loss for e

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './MNIST_LR' # Here you have to put your log directory
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

![](https://i.loli.net/2018/11/20/5bf39a64468dc.png)